## Getting the dataset

In [1]:
! kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
100%|██████████████████████████████████████| 80.9M/80.9M [00:14<00:00, 5.54MB/s]
100%|██████████████████████████████████████| 80.9M/80.9M [00:14<00:00, 6.03MB/s]


In [3]:
# specifying the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# extracting the compresssed dataset
from zipfile import ZipFile
dataset = '~/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
    zip.extractall()
    print('the dataset is extracted')

the dataset is extracted


## Importing the dependencies

In [13]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [14]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/abdullahraashid/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
#printing the stopwords in english
print(stopwords.words('english')) #stopwords are essentially the words which don't add significant meaning to your analysis, as sentiment can not be gazed from it alone.

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## Data Processing

In [16]:
# Loading the data from csv file to pandas dataframe
twitter_data = pd.read_csv('/Users/abdullahraashid/Documents/PythonML/TwitterSentimentAnalysis/training.1600000.processed.noemoticon.csv', encoding = 'ISO-8859-1')

In [17]:
# checking rows and columns
twitter_data.shape # 1.6 million tweets are rows, the 6 fields are the columnds, target, ids, date and all

(1599999, 6)

In [19]:
# print the first five rows
twitter_data.head() # Now it is accidentally assuming that the first tweet is the column names

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [20]:
# naming the cloumns and reading the dataset again
column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('/Users/abdullahraashid/Documents/PythonML/TwitterSentimentAnalysis/training.1600000.processed.noemoticon.csv', names = column_names, encoding = 'ISO-8859-1')
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [21]:
# Number of missing values in the dataset
twitter_data.isnull().sum()

target    0
ids       0
date      0
flag      0
user      0
text      0
dtype: int64

In [22]:
# checking the districution of target column, how many positive, neutral, and negative tweets are present
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

**Convert the target '4' to '1'**

In [25]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [26]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

**Stemming**: process of reducing a word to its Root word.
Example: actor, actress, acting = act

In [27]:
port_stem = PorterStemmer() # loads the instance

In [28]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ', content) #removes everything except letters since sentiment analysis
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split() #split everything and add it to a list, so if a tweet has 10 words those 10 individual words will be added to the list 
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] # if the word is not in stopwords, it will be stemmed
    stemmed_content = ' '.join(stemmed_content) # join everything 
    return stemmed_content

In [29]:
twitter_data['stemmed_text'] = twitter_data['text'].apply(stemming) # 50 minutes to complete the stemming

In [30]:
twitter_data.head()

,target,ids,date,flag,user,text,stemmed_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see


In [32]:
print(twitter_data['stemmed_text'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_text, Length: 1600000, dtype: object


In [33]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [34]:
# seperating the data(stemmed_text) and label(target)
X = twitter_data['stemmed_text'].values
y = twitter_data['target'].values

In [35]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


**Splitting the data to training and testing data**

In [37]:
# creating 4 arrays
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,stratify=y, random_state = 5) 

In [38]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [39]:
## Converting the textual data to numerical data: feature extraction, vectorizer

vectorizer = TfidfVectorizer()

X_train=vectorizer.fit_transform(X_train) # assigns value to words that have been repeated and ML model tries to correspond that word to either zero or 1
X_test=vectorizer.transform(X_test) # so you don't fit the test data as you need it to test your model

In [40]:
print(X_train) #all the words get some importance value

  (0, 274638)	0.45362265936406476
  (0, 169423)	0.4307460563004172
  (0, 124737)	0.5231197021850295
  (0, 153268)	0.4594366485420904
  (0, 93843)	0.3520625811633061
  (1, 283017)	0.30050337168173064
  (1, 181713)	0.37735485447514094
  (1, 331003)	0.5650798659991657
  (1, 39493)	0.1402323392193017
  (1, 66382)	0.2524858074543787
  (1, 426531)	0.1476075287094744
  (1, 272655)	0.1922328392689911
  (1, 249196)	0.11625131115897648
  (1, 438493)	0.17920320711015286
  (1, 33752)	0.3606951529447222
  (1, 429569)	0.3606951529447222
  (2, 377901)	0.5257232560652112
  (2, 310001)	0.6920301869860533
  (2, 400406)	0.21359272849862912
  (2, 346877)	0.4461921387303361
  (3, 427132)	0.5878957014210386
  (3, 2485)	0.38221253706572816
  (3, 127112)	0.3120936733238297
  (3, 78450)	0.3348573915248512
  (3, 102415)	0.4207438797374329
  :	:
  (1279996, 234412)	0.3960194379774884
  (1279996, 234103)	0.6001427974252357
  (1279997, 240545)	0.18863558609952527
  (1279997, 151762)	0.17104665114835857
  (1279997,

In [41]:
print(X_test)

  (0, 6647)	0.6383113115794222
  (0, 96204)	0.3883553643040905
  (0, 118629)	0.39959613334366184
  (0, 120039)	0.27350243321462087
  (0, 234671)	0.18241073412253994
  (0, 311854)	0.4171144373494487
  (1, 98)	0.5773211737315743
  (1, 39493)	0.2358289629722106
  (1, 78763)	0.1870233790393096
  (1, 88887)	0.4596247711759963
  (1, 138886)	0.21710676644564528
  (1, 171195)	0.1742967177513577
  (1, 258502)	0.2499920169561539
  (1, 259198)	0.25961146603053226
  (1, 358217)	0.18851425508300926
  (1, 409728)	0.272543871815237
  (1, 436990)	0.21823309985776707
  (2, 149798)	0.2758351274723409
  (2, 225636)	0.5431230906491545
  (2, 341209)	0.6190504487881319
  (2, 365683)	0.4956902588423947
  (3, 9486)	0.22502304383253255
  (3, 9735)	0.34469144514205413
  (3, 26529)	0.1987608273386633
  (3, 30294)	0.25839647540464467
  :	:
  (319997, 453603)	0.46427748955890213
  (319998, 79030)	0.3789882929445871
  (319998, 89548)	0.22640793411705282
  (319998, 93843)	0.1433460341233969
  (319998, 233897)	0.3137

## Training the Logistic Regression model

In [43]:
model = LogisticRegression(max_iter=1000)

In [44]:
model.fit(X_train,y_train) # maps to the desired targets

LogisticRegression(max_iter=1000)

## Model Evaluation

**Accuracy Score**

In [45]:
# Accuracy score on the training data
X_train_pred = model.predict(X_train)
training_data_accuracy=accuracy_score(y_train,X_train_pred)

In [46]:
print('accuracy score on the training data = ', training_data_accuracy) # 0.78068828125, 78% accuracy

accuracy score on the training data =  0.78068828125


In [47]:
# Accuracy score on the training data
X_test_pred = model.predict(X_test)
training_data_accuracy=accuracy_score(y_test,X_test_pred)
print('accuracy score on the test data = ', training_data_accuracy) # 0.772940625, 77%, so model is performing well, and no overfitting has taken place

accuracy score on the test data =  0.772940625


Model accuracy = 77.8%

## Saving the trained model

In [50]:
import pickle

In [51]:
filename = 'twitter_sentiment.sav'
pickle.dump(model,open(filename,'wb')) # dumps/writes the model in the filename 

## Using the saved model for future predictions

In [54]:
# loading the saved model
loaded_model = pickle.load(open('~/twitter_sentiment.sav','rb'))

In [56]:
X_new = X_test[45]
print(y_test[45])

prediction = loaded_model.predict(X_new)
print(prediction)

if (prediction[0] ==0):
    print('Negative Tweet')
else:
    print('Positive Tweet')

1
[1]
Positive Tweet
